In [29]:
import pandas as pd
news_data = pd.read_csv('Combined_News_DJIA.csv')


In [30]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords from NLTK (do this only once)
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# Load the dataset
data = pd.read_csv('Combined_News_DJIA.csv')

# Basic text preprocessing function
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove any URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words and word.isalpha()]
    
    return " ".join(filtered_tokens)

# Apply the preprocessing function to each headline column
for i in range(1, 26):  # Loop through all columns
    col_name = f'Top{i}'
    data[f'cleaned_{col_name}'] = data[col_name].astype(str).apply(preprocess_text)

print(data[['cleaned_Top1', 'cleaned_Top2', 'cleaned_Top3']].head())  # Just print first 3 cleaned columns as an example


[nltk_data] Downloading package stopwords to /Users/TrexK/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/TrexK/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


                                        cleaned_Top1  \
0  bgeorgia downs two russian warplanes countries...   
1  bwhy wont america nato help us wont help us he...   
2    bremember adorable sang opening ceremonies fake   
3     b us refuses israel weapons attack iran report   
4                  ball experts admit legalise drugs   

                                        cleaned_Top2  \
0                      bbreaking musharraf impeached   
1                  bbush puts foot georgian conflict   
2                     brussia ends georgia operation   
3  bwhen president ordered attack tskhinvali capi...   
4    bwar south osetia pictures made russian soldier   

                                        cleaned_Top3  
0  brussia today columns troops roll south osseti...  
1  bjewish georgian minister thanks israeli train...  
2               bif sexual harassment would children  
3    b israel clears troops killed reuters cameraman  
4  bswedish wrestler ara abrahamian throws away m..

In [34]:
# Assuming you have already processed and created cleaned columns as in the previous steps...

# Create a list of original columns names
original_cols = [f'Top{i}' for i in range(1, 26)]

# Drop the original columns
data = data.drop(columns=original_cols)

# Now, your DataFrame 'data' will only contain cleaned columns and other columns that were not dropped.
print(data.head())  # To verify the remaining columns


         Date  Label                                       cleaned_Top1  \
0  2008-08-08      0  bgeorgia downs two russian warplanes countries...   
1  2008-08-11      1  bwhy wont america nato help us wont help us he...   
2  2008-08-12      0    bremember adorable sang opening ceremonies fake   
3  2008-08-13      0       us refuses israel weapons attack iran report   
4  2008-08-14      1                  ball experts admit legalise drugs   

                                        cleaned_Top2  \
0                      bbreaking musharraf impeached   
1                  bbush puts foot georgian conflict   
2                     brussia ends georgia operation   
3  bwhen president ordered attack tskhinvali capi...   
4    bwar south osetia pictures made russian soldier   

                                        cleaned_Top3  \
0  brussia today columns troops roll south osseti...   
1  bjewish georgian minister thanks israeli train...   
2               bif sexual harassment would 

In [35]:
!pip install nltk

import nltk
from nltk.corpus import words

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 1.4 MB 6.2 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 5.0 MB/s  eta 0:00:01
     |████████████████████████████████| 294 kB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 382 kB 10.7 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 8.2 MB/s eta 0:00:011
  Using 

In [37]:
nltk.download('words')


[nltk_data] Downloading package words to /Users/TrexK/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [38]:
word_list = set(words.words())

def conditional_remove_b(text):
    # If the string starts with a "b" and the word without "b" isn't in the dictionary, remove the "b"
    if text.startswith('b') and text[1:].lower() not in word_list:
        return text[1:]
    return text


In [39]:
for i in range(1, 26):
    col_name = f'cleaned_Top{i}'
    data[col_name] = data[col_name].apply(conditional_remove_b)


In [42]:
data['combined_news'] = data.filter(like='cleaned_').apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(data['combined_news'])
y = data['Label']


In [44]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [45]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

In [46]:
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))


0.4723618090452261


In [61]:
def predict_headline_movement(headline):
    # Transform the headline using TF-IDF vectorizer
    transformed_headline = vectorizer.transform([headline])
    
    # Predict using the classifier
    prediction = clf.predict(transformed_headline)

    return "Up" if prediction[0] == 1 else "Down"

# Example usage
result = predict_headline_movement(input())
print(f"The stock is predicted to go {result}.")


The stock is predicted to go Up.
